In [64]:
import sympy
from generate_cpp_code import *

In [65]:
dhat = sympy.Symbol(r'\hat{d}')
d = sympy.Symbol('d')
dmin = sympy.Symbol('d_{\\text{min}}')

In [66]:
def barrier(d, dhat):
    return -(d - dhat)**2 * sympy.log(d/dhat)


def normalized_barrier(d, dhat):
    return -(d/dhat - 1)**2 * sympy.log(d/dhat)


def physical_barrier(d, p_dhat):
    return dhat * normalized_barrier(d, p_dhat)


def log_sq_barrier(d, dhat):
    return (d - dhat)**2 * sympy.log(d/dhat)**2


display(
    barrier(d, dhat),
    normalized_barrier(d, dhat),
    physical_barrier(d, dhat),
    log_sq_barrier(d, dhat)
)

-(-\hat{d} + d)**2*log(d/\hat{d})

-(-1 + d/\hat{d})**2*log(d/\hat{d})

-\hat{d}*(-1 + d/\hat{d})**2*log(d/\hat{d})

(-\hat{d} + d)**2*log(d/\hat{d})**2

In [67]:
(physical_barrier(d, dhat) / barrier(d, dhat)).simplify()

1/\hat{d}

In [68]:
(physical_barrier(d**2, dhat**2) / barrier(d**2, dhat**2)).simplify()

\hat{d}**(-3)

In [69]:
d_param = d**2 - dmin**2
dhat_param = 2 * dmin * dhat + dhat**2
(physical_barrier(d_param, dhat_param) / barrier(d_param, dhat_param)).simplify()

1/(\hat{d}*(\hat{d} + 2*d_{\text{min}})**2)

## Code Generation

In [70]:
print(generate_code(normalized_barrier(
    d, dhat)).replace(r"\hat{d}", "dhat"))

const auto t0 = d/dhat;
return -std::pow(1 - t0, 2)*std::log(t0);


In [71]:
print(generate_code(normalized_barrier(d, dhat).diff(
    d)).replace(r"\hat{d}", "dhat"))

const auto t0 = 1.0/dhat;
const auto t1 = d*t0;
const auto t2 = 1 - t1;
return t2*(2*t0*std::log(t1) - t2/d);


In [72]:
print(generate_code(normalized_barrier(d, dhat).diff(
    d).diff(d)).replace(r"\hat{d}", "dhat"))

const auto t0 = 1.0/dhat;
const auto t1 = d*t0;
const auto t2 = 1 - t1;
return 4*t0*t2/d + std::pow(t2, 2)/std::pow(d, 2) - 2*std::log(t1)/std::pow(dhat, 2);


In [73]:
print(generate_code(log_sq_barrier(
    d, dhat).diff(d).diff(d)).replace(r"\hat{d}", "dhat"))

const auto t0 = std::log(d/dhat);
const auto t1 = dhat - d;
const auto t2 = std::pow(t1, 2)/std::pow(d, 2);
return 2*std::pow(t0, 2) - 2*t0*t2 + 2*t2 - 8*t0*t1/d;
